# utilisation d'un catalogue Kerchunk pour ouvrir l'archive Gamar

In [1]:
%%time 
import json
import fsspec
import xarray as xr

CPU times: user 357 ms, sys: 144 ms, total: 501 ms
Wall time: 1.12 s


In [2]:
#small_ds=ds.isel(x_rho =slice(390,452), y_rho =slice(400,430) )[['temp','salt','zeta']].chunk(chunks={"time_counter":"10M"}).compute()
small_ds=xr.open_zarr('/Users/todaka/data/RIOMAR/very_small.zarr')# ds.isel(time_counter=0, s_rho=0)[['temp']].compute()
small_ds

<xarray.Dataset> Size: 957kB
Dimensions:       (s_rho: 40, y_rho: 40, x_rho: 72)
Coordinates:
  * s_rho         (s_rho) float32 160B -0.9875 -0.9625 ... -0.0375 -0.0125
  * y_rho         (y_rho) float32 160B 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
  * x_rho         (x_rho) float32 288B 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    nav_lat_rho   (y_rho, x_rho) float32 12kB dask.array<chunksize=(40, 72), meta=np.ndarray>
    nav_lon_rho   (y_rho, x_rho) float32 12kB dask.array<chunksize=(40, 72), meta=np.ndarray>
    time_counter  datetime64[ns] 8B ...
    time_instant  datetime64[ns] 8B ...
Data variables:
    salt          (s_rho, y_rho, x_rho) float32 461kB dask.array<chunksize=(40, 40, 72), meta=np.ndarray>
    temp          (s_rho, y_rho, x_rho) float32 461kB dask.array<chunksize=(40, 40, 72), meta=np.ndarray>
    zeta          (y_rho, x_rho) float32 12kB dask.array<chunksize=(40, 72), meta=np.ndarray>
Attributes: (12/42)
    name:           GAMAR_GLORYS_1h_inst
    description:    Created by xios
    Conventions:    CF-1.6
    title:          GAMAR_GLORYS
    rst_file:       croco_rst.nc
    grd_file:       croco_grd.nc
    ...             ...
    gamma2_expl:    Slipperiness parameter
    x_sponge:       0.0
    v_sponge:       0.0
    sponge_expl:    Sponge parameters : extent (m) & viscosity (m2.s-1)
    SRCS:           main.F step.F read_inp.F timers_roms.F init_scalars.F ini...
    CPP-options:    REGIONAL GAMAR MPI TIDES OBC_WEST OBC_NORTH XIOS USE_CALE...

In [3]:
import hvplot.xarray
small_ds.temp.isel(s_rho=0).hvplot.quadmesh(y='nav_lat_rho', x='nav_lon_rho',  ylim=(46.9, 47.5))#,coastline=True)#),geo=True,datashader=False)#,coastline=True)


:QuadMesh   [nav_lon_rho,nav_lat_rho]   (potential temperature)

In [4]:
import numpy as np




# Make sure lon/lat are aligned with temp and NaNs are dropped consistently
tmp = small_ds[["temp", "nav_lon_rho", "nav_lat_rho"]].isel(s_rho=0).stack(point=("y_rho","x_rho"))
tmp = tmp.dropna(dim="point").compute()  # drops points where any variable is NaN
lon = tmp["nav_lon_rho"].values.astype(np.float64)
lat = tmp["nav_lat_rho"].values.astype(np.float64)
data= tmp.temp.values.astype(np.float64)
data,lon,lat

(array([15.14239693, 15.1304245 , 15.1210022 , ..., 14.22236729,
        13.56026936, 12.48732758], shape=(2059,)),
 array([-2.86666656, -2.85333323, -2.83999991, ..., -2.54666662,
        -2.5333333 , -2.51999998], shape=(2059,)),
 array([46.99822235, 46.99822235, 46.99822235, ..., 47.35172653,
        47.35172653, 47.35172653], shape=(2059,)))

In [5]:
device = "cpu"
from regrid_to_healpix.regrid_to_healpix_bilinear import Set
#nr = Set(lon_deg=lon, lat_deg=lat, level=level, device=device, threshold=0.7, ellipsoid="WGS84") 
level = 13
nr = Set(lon_deg=lon, lat_deg=lat,level=level, device=device, threshold=0.5, ellipsoid="WGS84")
#from regrid_to_healpix.regrid_to_healpix_GEN import Set
#nr = Set(lon_deg=lon, lat_deg=lat,  Npt = 2, level=level, device=device, ellipsoid="WGS84") 
#from regrid_to_healpix.regrid_to_healpix_nearest import Set
#level = 12
#nr = Set(lon_deg=lon, lat_deg=lat, level=level, device=device, ellipsoid="WGS84") 

#from regrid_to_healpix.regrid_to_healpix_psf import Set
#level = 13
#nr = Set(lon_deg=lon, lat_deg=lat, level=level, device=device, threshold=0.7, ellipsoid="WGS84") 


/Users/todaka/python/git/regrid_to_healpix/regrid_to_healpix/regrid_to_healpix_bilinear.py:74: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/SparseCsrTensorImpl.cpp:51.)
  self.M  = M_coo.to_sparse_csr()


In [6]:

# puis:
data_healpix=nr.transform(data,lam=0.1)  # selon ton API
data_healpix_invert = nr.invert(data_healpix)
cell_ids = nr.get_cell_ids()
#hval.shape#,rval.shape
#cell_ids , data_healpix.shape, data_healpix_invert.shape

In [7]:
import xdggs
def make_xdggs_dataarray_from_cell_ids(data, cell_ids, level=15,  name="da",):
    """
    Build an xdggs-compatible DataArray:
      - dim/coord: 'cell_ids'
      - values: factor (same for all cells)
      - coord attrs required by xdggs
    """
    cell_ids = np.asarray(cell_ids, dtype=np.int64)

    da = xr.DataArray(
        data,
        dims=("cell_ids",),
        coords={"cell_ids": ("cell_ids", cell_ids)},
        name=name,
    )

    da["cell_ids"].attrs.update(
        {
            "grid_name": "healpix",
            "level": int(level),
            "indexing_scheme": "nested",
            "ellipsoid":"WGS84"
        }
    )
    return da.pipe(xdggs.decode).dggs.assign_latlon_coords()
make_xdggs_dataarray_from_cell_ids(data_healpix, cell_ids,level=level).dggs.explore(alpha=0.3)

In [11]:
data_healpix.max() - data.max() , data_healpix.min() - data.min(), data_healpix.mean() - data.mean(),

(np.float64(-0.005251869245245899),
 np.float64(0.0),
 np.float64(-0.004745451393270983))

## Catalogue Kerchunk local, ouverture avec open_dataset pour usage_courant

## Catalogue Kerchunk local, ouverture avec open_VIRTUAL_dataset
(pour analyse des manifests, réécriture des paths, divers..)

## Catalogue kerchunk http

- Note Fred : ca marche a l'exterieur d'ifremer, mais pas depuis le hpc ifremer par exemple (timeout!)...
- Avec Colab, ca fonctionne : https://colab.research.google.com/drive/18MWwjhNEaWMdhhcWun_3sfS6uZYJishs#scrollTo=oRdXBDrF34sy

TODO ? (là c'est moche! mais ca marche)
Utilisation d'un script shell pour modifier les paths des fichiers, et en faire des urls valides au lieu des chemins locaux. Voir comment faire ca directement en python avec l'api kerchunk si possible